# Redes Neurais Artificiais - Multilayer Perceptron

**Acompanhar explicação pelos slides da teoria.**

### Criação dos dados:

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

todos_dados, todos_alvos = make_blobs(n_samples=400, n_features=2, centers=3, cluster_std=.99, shuffle=True)

alvos_map = map(lambda label: [1] if label == 2 else [label], todos_alvos)
todos_alvos = list(alvos_map)

dados, dados_teste, alvos, alvos_teste = train_test_split(todos_dados, todos_alvos, test_size=.2, random_state=44)

In [ ]:
# import numpy as np

# dados = np.array(
#     [[5.7, 7.5], [9.3, 5.6], [8.8, 6.9], [6.8, 9.2], [7.9, 9.1],
#     [8.8, 5.6], [5.6, 7.8], [8.8, 9.0], [7.6, 5.9], [4.9, 8.1],
#     [6.6, 4.5], [5.2, 7.2], [1.2, 1.2], [2.4, 1.6], [4.7, 2.5],
#     [0.9, 3.1], [2.6, 4.1], [1.8, 2.0], [4.1, 2.8], [1.8, 1.6],
#     [1.9, 3.1], [0.8, 3.2], [1.8, 2.9], [3.3, 1.6], [3.7, 2.5]
#   ])

# alvos = np.array([[1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]])

In [ ]:
import matplotlib.pyplot as plt

plt.title("'dados' separado por 'alvos'")
plt.scatter(dados[:, 0], dados[:, 1], c = alvos)
plt.show()

### Modelando rede MLP:

In [ ]:
import math
import numpy as np

class MLP:
    def __init__(self, taxa_aprendizado: float, tolerancia_taxa_erro: float, neuronios: list[int]):
        self.taxa_aprendizado = taxa_aprendizado
        self.tolerancia_taxa_erro = tolerancia_taxa_erro
        self.neuronios = neuronios

    def __sigmoid(self, valor):
        '''
            Calcula a sigmoid de um valor.
        '''
        return (1/(1+math.e**(-valor)))

    def __deriv_sigmoid(self, valor):
        '''
            Calcula a derivada da função sigmoid.
        '''
        sig = self.__sigmoid(valor)
        return sig*(1 - sig)

    def __ativacao(self, valor: float) -> int:
        '''
            Função de ativação.
        '''
        return self.__sigmoid(valor)
    
    def __deriv_ativacao(self, valor):
        '''Calcular a derivada da função de ativação'''
        return self.__deriv_sigmoid(valor)
        
    def __predicao(self, valores: float, pesos: float) -> float:
        '''
            Função de predição.
            Realiza a multiplicação matricial entre as entradas e os pesos somado ao bias proporcional.
        '''
        return np.dot(valores, pesos).reshape(1, -1)
    
    def __avaliacao(self, valor_alvo: float, valor_saida: float) -> float:
        '''
            Função de avaliação.
            Calcula a diferença entre o valor alvo e o valor de saida.
        '''
        return (valor_alvo - valor_saida)
    
    def treinar(self, dados: list[list[float]], alvos: list[float]):
        '''
            Treino da rede MLP.
            Define aleatoriamente os pesos para cada camada.
            Enquanto a taxa de erro for maior que o aceitável continua o processo.
        '''
        self.bias = -1

        self.peso_1 = np.random.random((dados.shape[1] + 1, self.neuronios[0]))
        self.peso_2 = np.random.random((self.neuronios[0], self.neuronios[1]))
        self.peso_3 = np.random.random((self.neuronios[1], self.neuronios[2]))

        epoca = 0
        self.erros = []

        while True:

            erro_medio_quadratico_epoca = 0

            for dado, alvo in zip(dados, alvos):

                # Feed-foward
                entradas = np.insert(dado, 0, self.bias)

                i1 = self.__predicao(entradas, self.peso_1)
                y1 = self.__ativacao(i1)

                i2 = self.__predicao(y1, self.peso_2)
                y2 = self.__ativacao(i2)

                i3 = self.__predicao(y2, self.peso_3)
                y3 = self.__ativacao(i3)
                
                erro = self.__avaliacao(alvo, y3)
                erro_medio_quadratico_epoca += erro ** 2

                # Backpropagation
                delta3 = (alvo - y3) * self.__deriv_ativacao(i3)
                self.peso_3 += self.taxa_aprendizado * np.dot(y2.T, delta3)

                delta2 = np.dot(delta3, self.peso_3.T) * self.__deriv_ativacao(i2)
                self.peso_2 += self.taxa_aprendizado * np.dot(y1.T, delta2)

                delta1 = np.dot(delta2, self.peso_2.T) * self.__deriv_ativacao(i1)
                self.peso_1 += self.taxa_aprendizado * np.dot(entradas.reshape(1, -1).T, delta1)

            erro_medio_quadratico_epoca = erro_medio_quadratico_epoca / len(dados)
            taxa_erro = abs((np.inf if not len(self.erros) else self.erros[-1]) - erro_medio_quadratico_epoca)
            print(f'Época: {epoca}\n\t- Erro Quadratico Medio: {erro_medio_quadratico_epoca}\n\t- Taxa Erro: {taxa_erro}')
            if taxa_erro <= self.tolerancia_taxa_erro:
                break

            self.erros.append(erro_medio_quadratico_epoca[0])
            epoca += 1

    def testar(self, dados: list[list[float]]) -> list[float]:
        '''
            Testa a rede treinada.
            Dado os dados, submete-os à rede para predição da saída.
        '''
        saidas = []
        for dado in dados:
            entradas = np.insert(dado, 0, self.bias)

            i1 = self.__predicao(entradas, self.peso_1)
            y1 = self.__ativacao(i1)

            i2 = self.__predicao(y1, self.peso_2)
            y2 = self.__ativacao(i2)

            i3 = self.__predicao(y2, self.peso_3)
            y3 = self.__ativacao(i3)

            saida_aproximada = np.rint(y3)

            saidas.append(int(saida_aproximada))

        return saidas

### Treinando a rede:

In [ ]:
rede_mlp = MLP(taxa_aprendizado=0.01, tolerancia_taxa_erro=0, neuronios=[4, 3, 1])

rede_mlp.treinar(dados=dados, alvos=alvos)

### Testando a rede:

In [ ]:
# dados_teste = np.array([[-2,-7.5], [2, -10], [7, -9]])

saidas_teste = rede_mlp.testar(dados=dados_teste)

plt.title("'dados' e 'alvos' com as 'saidas_teste'")
plt.scatter(dados[:, 0], dados[:, 1], c = alvos)
plt.scatter(dados_teste[:, 0], dados_teste[:, 1], c = saidas_teste, marker = "*", edgecolors='black', s=250)
plt.show()

### Métricas da rede treinada:

Valores de erro:

In [ ]:
plt.title("Erro por época.")
plt.plot(rede_mlp.erros)
plt.show()

Pesos da rede:

In [ ]:
print(f'Pesos camada 1: {rede_mlp.peso_1}')
print(f'Pesos camada 2: {rede_mlp.peso_2}')
print(f'Pesos camada 2: {rede_mlp.peso_3}')